# BEST FIRST SEARCH

In [1]:
%run Piyush_Lohokare_AI_CW1_230238819.ipynb

In [2]:
'''
We are calculating the heuristic of the current node and returning the heuristic value
How I am calculating the heuristic:
- Get the value of the current station's zone and the goal station's zone from the zone_dict
- Say, current station's zone is Zone 1 and Goal Stations's Zone is Zone c. So, values of the zones, from the heu_dict, will 
be 10 and 90 respectively
- Get the absolute value of the difference between the goal zone and current zone if the zones are different so that the 
difference is non-negative
- If the zones of the current and goal station are same but the current station != goal station, return 1 by default, which will be 
the lowest heuristic value in the list and this node will be popped from the list next to generate its neighbors
- If the current station == goal station, return 0 by default as we have reached the goal
'''
def calc_heuristic(curr, goal):
    heuristic_dist = []
    ret = 0

    # If zone_dict[curr] is True
    if len(zone_dict[curr]):
      # Iterate through the zone_dict for the current station
        for zone_curr in zone_dict[curr]:
            # Iterate through the zone_dict for the goal station
            for zone_goal in zone_dict[goal]:
                # If the current station == goal station, return 0 by default as we have reached the goal
                if curr == goal:
                    ret = 0
                    return ret
                '''
                If the zones of the current and goal station are same but the current station != goal station, return 1 by
                default, which will be the lowest heuristic value in the list
                '''
                if zone_goal == zone_curr:
                    ret = 1
                    return ret
                else:
                    '''
                    If the zones of current and goal are different and the current station is not the goal station, we calculate
                    the distance between the zones and assign that as the heuristic cost for the current station
                    '''
                    heuristic_dist.append(abs(int(zone_goal) - int(zone_curr)))

    '''
    As we are appending all the heuristics for the current station in the heuristic_dist list, we'll use the minimum of the heuristic value.
    If the current station is in two zones or the goal station is in two zones we use the lowest heuristic as the travelling 
    distance between the stations is not changing.
    Only the zone is different.
    '''
    return min(heuristic_dist)

#### My motivation to use this heuristic
- I needed a value for each of the zones to be able to calculate the heuristics of each station. However, it is really difficult to assign a value to the goal station each time we generate a neighbor. So, it was efficient to assign each zone a fixed value. This value has no other purpose than helping in calculating the heuristic.
  
- The values can be taken as [1, 2, 3, …, 10] or [20, 40, …, 200]. However, my preferred way was to consider [10, 20, …, 100]. Once I had this setup, getting the heuristic was just calculating the absolute difference between the goal zone and the neighbor’s zone. The algorithm will sort the BFS_list as per the lowest heuristic in it so that the lowest heuristic station is at the start of the list, which will be popped next. 

- Now, there are 2 exceptions while calculating the absolute difference between the goal zone and the neighbor’s zone. It can happen that the zones of the neighbor and the goal are same but the neighbor station is not the goal station. In this case, I’m returning the value 1 by default which will be the lowest heuristic value in the list and this node will be popped from the list next to generate its neighbors. 

- If the neighbor and the goal are same, I’m by default returning the heuristic as 0 as we have reached the goal node and then terminate the loop. 

In [3]:
'''
I HAVE TAKEN THIS CODE FROM THE LAB 2 SOLUTION FROM THE FUNCTION "def construct_path_from_root(node, root):" 
UNDER THE HEADING Solution DFS -- variation 1 and tweaked it as needed
'''

def construct_path_from_root_BFS(node, root):
    path_from_root = [node['label']]
    line = set()
    line.add(node['line'])
    while node['parent']:
        node = node['parent']
        path_from_root = [node['label']] + path_from_root
        if node['line'] == None:
            break
        line.add(node['line'])

    return path_from_root, line#, cost_from_root

In [4]:
def best_first_search(initial, goal, calc_exploration_cost):
    BFS_list = [{'label':initial, 'parent':None, 'h(n)':None, 'line':None, 'cost':0}]
    explored_nodes = [initial]
    num_explored_nodes = 0
    neighbors = []
    line_list = []

    while BFS_list:
        curr_station = BFS_list.pop(0) # Pop the first node from the list as it is the node with the lowest heuristic
        num_explored_nodes += 1

        '''
        Check if the heuristic of the current node is 0.
        If it is 0, it means that we have reached the goal node and we have found our path.
        '''
        if curr_station['h(n)'] == 0:
            if calc_exploration_cost:
                print('\nNumber of explorations = {}'.format(num_explored_nodes))
            return curr_station

        '''
        Generate neighbors:
        station_dict is the dictionary. We are fetching the the values from the dictionary where the
        key is "curr_station['label']"
        curr_station['label'] is the station name that we just popped from the LIFO_list
        So, if the curr_station['label'] is Euston, we'll get all the neighbors of Euston from the station_dict
        '''
        for ind, stn in enumerate(station_dict[curr_station['label']]):
            neighbors.append(stn)

        for child_label in neighbors:
            child = {'label':child_label[0], 'parent': curr_station, 'h(n)':calc_heuristic(child_label[0], goal), 'line':child_label[2], 'cost':(child_label[1]+curr_station['cost'])}
            if child_label[0] not in explored_nodes:
                BFS_list.append(child)
                '''
                Here, we are sorting the BFS_list basis the heuristic. So, the lowest heuristic station will be at the start of the
                list and the highest heuristic station will be at the end of the list.
                '''
                BFS_list = sorted(BFS_list, key = itemgetter('h(n)'))
                explored_nodes.append(child_label[0])

    return None

In [5]:
print('\n======================================================================================\n')
print('========================== Best First Search Implementation ==========================\n')
for stn in test_paths:
    print('Start Station: ', stn[0])
    print('End Station: ', stn[1])
    solution = best_first_search(stn[0], stn[1], True)
    root_path, line = construct_path_from_root_BFS(solution, stn[0])

    print('\nPath taken:', root_path)
    print('\nLines used:', line)
    print('\nCost of travel:', solution['cost'])
    print('=================================')



========================== Best First Search Implementation ==========================

Start Station:  Euston
End Station:  Victoria

Number of explorations = 21

Path taken: ['Euston', 'Warren Street', 'Oxford Circus', 'Green Park', 'Victoria']

Lines used: {'Victoria', 'Northern'}

Cost of travel: 7
Start Station:  Canada Water
End Station:  Stratford

Number of explorations = 25

Path taken: ['Canada Water', 'Canary Wharf', 'North Greenwich', 'Canning Town', 'West Ham', 'Stratford']

Lines used: {'Jubilee'}

Cost of travel: 15
Start Station:  New Cross Gate
End Station:  Stepney Green

Number of explorations = 17

Path taken: ['New Cross Gate', 'Surrey Quays', 'Canada Water', 'Rotherhithe', 'Wapping', 'Shadwell', 'Whitechapel', 'Stepney Green']

Lines used: {'District', 'East London'}

Cost of travel: 14
Start Station:  Ealing Broadway
End Station:  South Kensington

Number of explorations = 22

Path taken: ['Ealing Broadway', 'West Acton', 'North Acton', 'East Acton', 'White Cit